In [1]:
from pyspark.sql import SparkSession
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization, Embedding, LSTM, Dense, Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
# 2. Spark Session and Load DataFrame
spark = SparkSession.builder.appName("RNN_Text_Classification").getOrCreate()

25/04/20 02:28:03 WARN Utils: Your hostname, Sameers-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.29.34 instead (on interface en0)
25/04/20 02:28:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/20 02:28:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


25/04/20 03:38:03 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 906434 ms exceeds timeout 120000 ms
25/04/20 03:38:03 WARN SparkContext: Killing executors is not supported by current scheduler.
25/04/20 03:38:06 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$